# Freelancer Timesheet Analyzer
Status: Version 2 not yet complete: reporting functions need to be re-written after moving from nested-lists to pandas dataframe.

## Version 2
1. Only Excel spreadsheets (.xlsx) formats are accepted.
1. Data is loaded into dataframes.
2. These columns are required:
 * Project (string) (such as Customer name or school class name)
 * Date (date)
 * Hours (float) 
3. These columns are optional:
 * Billable (Null, or any value)
3. All other columns loaded (available for customizations) but ignored.
3. Filters include:
 * Project 
 * Date range
 * Billable (if exists)
4. Output includes:
 * Print of total hours (decimal format)
 
## Version 1
1. Only CSV (.csv) are accepted.
2. Data is loaded into nested lists.

 

In [33]:
import pandas as pd

from __future__ import division

from tkinter import filedialog
import csv
from pprint import pprint
import datetime

#### Load Data

In [34]:
# Prompt for datafile 
#datafile = filedialog.askopenfilename(\
#    title='Select Timesheet CSV', \
#    filetypes=[("CSV","*.csv"),("All files","*.*")])
 
# hardcoded for testing TODO: go back to prompting for file
datafile = ('data/example.xlsx')

# load data
df = pd.read_excel(datafile) 

# TODO: add requirements validation

#### Clean Data

In [35]:
# Convert Billable: any value == 1; Null == 0
df['Billable'] = (df['Billable'].notnull()).astype('int')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 10 columns):
Date        26 non-null datetime64[ns]
Project     26 non-null object
Category    26 non-null object
Billable    26 non-null int64
Included    0 non-null float64
Details     26 non-null object
Start       23 non-null object
Stop        23 non-null object
Time        26 non-null object
Hours       26 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 2.1+ KB


In [37]:
df.head()

,Date,Project,Category,Billable,Included,Details,Start,Stop,Time,Hours
0,2018-01-01,Customer 1,Document,1,NaN,Document site architecture,08:40:00,09:25:00,00:45:00,0.750000
1,2018-01-02,Customer 1,Develop,1,NaN,Landing page changes,07:30:00,07:43:00,00:13:00,0.216667
2,2018-01-02,Customer 1,Document,1,NaN,Document site architecture,07:43:00,08:30:00,00:47:00,0.783333
3,2018-01-03,Customer 1,Develop,1,NaN,Landing page changes,08:30:00,09:50:00,01:20:00,1.333333
4,2018-01-04,Customer 1,Develop,1,NaN,Draft documentation,11:50:00,14:15:00,02:25:00,2.416667


In [38]:
def prompt_filters():
    # Print report, prompting for filters. TODO: improve

    project = input('Project: ') or None
    start_date = input('Start Date: ') or '01/01/1000'
    end_date = input('Start Date: ') or '12/31/9999'
    rate = input('Hourly Rate: ') or 60

    return (project, start_date, end_date, float(rate))
        

In [39]:
def filter_df(this_df, project=None, start_date=None, end_date=None, rate=None):
    '''
    Return dataframe of filtered date, with Billable NonBillable Hours separated.
    
    Input:  string (project name)
            string (date range start)
            string (date range end)
    Output: None
    Return: dataframe
    '''
    ## Build query
    # if no project specified, then set Project != None to return all 
    # TODO consider logic when there is no project name intentionally
    if not project:
        proj_equal = '!=' 
    else:
        proj_equal = '=='
        
    column =    ['Project',  'Date',     'Date']
    equal =     [proj_equal, '>=',       '<']
    condition = [project,    start_date, end_date]
    
    query = ' & '.join(f'{i} {j} {repr(k)}' for i, j, k in zip(column, equal, condition))
    print(query)
       
    return this_df.query(query)

In [40]:
project, start_date, end_date, rate = prompt_filters()
    
print('\nProject: {}\nStart Date: {}\nEnd Date: {}\nRate: {}'\
      .format(project, start_date, end_date, rate))

Project: 
Start Date: 
Start Date: 
Hourly Rate: 

Project: None
Start Date: 01/01/1000
End Date: 12/31/9999
Rate: 60.0


In [41]:
billing_df = filter_df(df, None, "2018-01-04", "2018-01-10" )

Project != None & Date >= '2018-01-04' & Date < '2018-01-10'


In [42]:
billing_df

,Date,Project,Category,Billable,Included,Details,Start,Stop,Time,Hours
4,2018-01-04,Customer 1,Develop,1,NaN,Draft documentation,11:50:00,14:15:00,02:25:00,2.416667
5,2018-01-05,Customer 1,Proj Mgmt,0,NaN,Prepare for meeting,NaN,NaN,00:30:00,0.500000
6,2018-01-08,Customer 1,Document,1,NaN,Infrastructure doc and deliver,NaN,NaN,02:00:00,2.000000
7,2018-01-09,Customer 2,Develop,1,NaN,portfolio,01:15:00,03:30:00,02:15:00,2.250000
8,2018-01-09,Customer 2,Develop,1,NaN,portfolio,06:20:00,06:50:00,00:30:00,0.500000


In [51]:
billing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 4 to 8
Data columns (total 10 columns):
Date        5 non-null datetime64[ns]
Project     5 non-null object
Category    5 non-null object
Billable    5 non-null int64
Included    0 non-null float64
Details     5 non-null object
Start       3 non-null object
Stop        3 non-null object
Time        5 non-null object
Hours       5 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 440.0+ bytes


In [57]:
billing_df[billing_df['Billable'] == 1]

,Date,Project,Category,Billable,Included,Details,Start,Stop,Time,Hours
4,2018-01-04,Customer 1,Develop,1,NaN,Draft documentation,11:50:00,14:15:00,02:25:00,2.416667
6,2018-01-08,Customer 1,Document,1,NaN,Infrastructure doc and deliver,NaN,NaN,02:00:00,2.000000
7,2018-01-09,Customer 2,Develop,1,NaN,portfolio,01:15:00,03:30:00,02:15:00,2.250000
8,2018-01-09,Customer 2,Develop,1,NaN,portfolio,06:20:00,06:50:00,00:30:00,0.500000


In [65]:
print('Total Hours: {:4f}'.format(billing_df['Hours'].sum()))
print('Non-Billable Hours: {:.4f}'.format( \
        billing_df[billing_df['Billable'] == 0]['Hours'].sum()))
print('Billable Hours: {:.4f}'.format( \
        billing_df[billing_df['Billable'] == 1]['Hours'].sum()))
print('Hourly Rate: ${:.2f}'.format(rate))
print('Billable Amount: ${:.2f}'.format( \
        billing_df[billing_df['Billable'] == 1]['Hours'].sum() * rate))

Total Hours: 7.666667
Non-Billable Hours: 0.5000
Billable Hours: 7.1667
Hourly Rate: $60.00
Billable Amount: $430.00
